In [ ]:
# 
# 
# MPAS DYAMOND has something different about time... 


from pathlib import Path
import numpy as np
import xarray as xr

# SET NECESSARY INPUT AND OUTPUT PATHS
dataloc = Path("/glade/campaign/mmm/wmr/fjudt/projects/dyamond_1/3.75km")
datafils = sorted(dataloc.glob("history.2016*.nc")) # note: 62GB per file

print(f"Identified {len(datafils)} files to remap to healpix and save to zarr.")

print(f"TEST TEST TEST --- REDUCE TO 2 FILES ---- ")
datafils = datafils[0:2]

# mesh description (maybe)
meshloc = Path("/glade/campaign/mmm/wmr/skamaroc/NSC_2023")
meshfil = meshloc / "x1.41943042.static.nc"


Identified 321 files to remap to healpix and save to zarr.
TEST TEST TEST --- REDUCE TO 2 FILES ---- 


In [6]:
import pandas as pd

In [64]:
def pre_proc_mpas_file(datafil, meshfil):
    ds_mpas = xr.open_dataset(datafil, engine='netcdf4', mask_and_scale=True, chunks={'Time': 'auto'})

    if len(ds_mpas['Time']) > 1:
        print("Multiple times detected... will probably break preprocessor")

    if isinstance(meshfil, xr.Dataset):
        ds_static = meshfil
    elif isinstance(meshfil, Path):
        ds_static = xr.open_dataset(meshfil)
    else:
        raise ValueError("meshfil needs to be a dataset or a path")

    # Use a fixed reference date for all files
    ref_date = '2000-01-01 00:00:00'  # Or any other suitable fixed date

    # Clean and convert xtime strings
    time_str = ds_mpas.xtime.astype(str).values.astype('U').ravel()
      
    print(f"{time_str = }")

    # Remove extra whitespace and handle empty strings
    time_str = [x.strip() for x in time_str]
    time_str = [x.replace("_", " ") for x in time_str]
      
    if isinstance(time_str, np.ndarray) or isinstance(time_str, list):
        time_str = "".join(time_str)

    print(f"{time_str = }")

    # Convert to datetime
    # change coordinate (and index) from "Time" to "time"
    time_coord = pd.to_datetime(time_str)

    print(time_coord)

    # Calculate hours since reference date for the coordinate values
    hours_since = (time_coord - pd.Timestamp(ref_date)) / pd.Timedelta('1h')
    if isinstance(hours_since, xr.DataArray):
        hours_since = hours_since.values
    elif isinstance(hours_since, float):
        hours_since = np.array([hours_since,])

    print(hours_since)
    print(f"{hours_since.shape}")

    # Create time coordinate with specific encoding
    time_var = xr.DataArray(
        hours_since,
        dims='Time',
        name='time',
        attrs={'long_name': 'time', 
               'axis': 'T',
               'reference_date': ref_date},
               )
    time_var.encoding = {
        'dtype': 'float64',
        'units': f'hours since {ref_date}',
        'calendar': 'standard',
        '_FillValue': None
    }
    
    ds_mpas_new = ds_mpas.assign_coords(time=('Time', hours_since))
    # ds_mpas_new = ds_mpas.assign_coords(time=('Time', time_var.data))
    
    ds_mpas_new = ds_mpas_new.swap_dims({"Time":"time"})

    # Find variables with dtype 'S64'
    s64_vars = [var for var in ds_mpas_new.variables if ds_mpas_new[var].dtype == 'S64']
    print(f"Variables with S64 dtype: {s64_vars}")

    # Drop these variables from the dataset
    ds_mpas_clean = ds_mpas_new.drop_vars(s64_vars)

    return ds_mpas_clean



In [4]:
fil = datafils[0]
# mesh description (maybe)
meshloc = Path("/glade/campaign/mmm/wmr/skamaroc/NSC_2023")
meshfil = meshloc / "x1.41943042.static.nc"

ds_static = xr.open_dataset(meshfil)


In [65]:
data = pre_proc_mpas_file(fil, ds_static)

time_str = array(['2', '0', '1', '6', '-', '0', '8', '-', '0', '1', '_', '0', '0',
       ':', '0', '0', ':', '0', '0', ' ', ' ', ' ', ' ', ' ', ' ', ' ',
       ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ',
       ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ',
       ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
      dtype='<U1')
time_str = '2016-08-01 00:00:00'
2016-08-01 00:00:00
[145368.]
(1,)
Variables with S64 dtype: []


In [66]:
data

<xarray.Dataset> Size: 179GB
Dimensions:                 (time: 1, StrLen: 64, nCells: 41943042,
                             nVertLevels: 75, nVertLevelsP1: 76, nSoilLevels: 4)
Coordinates:
  * time                    (time) float64 8B 1.454e+05
Dimensions without coordinates: StrLen, nCells, nVertLevels, nVertLevelsP1,
                                nSoilLevels
Data variables: (12/25)
    xtime_old               (time, StrLen) |S1 64B dask.array<chunksize=(1, 64), meta=np.ndarray>
    uReconstructZonal       (time, nCells, nVertLevels) float32 13GB dask.array<chunksize=(1, 762601, 1), meta=np.ndarray>
    uReconstructMeridional  (time, nCells, nVertLevels) float32 13GB dask.array<chunksize=(1, 762601, 1), meta=np.ndarray>
    w                       (time, nCells, nVertLevelsP1) float32 13GB dask.array<chunksize=(1, 748983, 1), meta=np.ndarray>
    temperature             (time, nCells, nVertLevels) float32 13GB dask.array<chunksize=(1, 762601, 1), meta=np.ndarray>
    pressure                (time, nCells, nVertLevels) float32 13GB dask.array<chunksize=(1, 762601, 1), meta=np.ndarray>
    ...                      ...
    vegfra                  (time, nCells) float32 168MB dask.array<chunksize=(1, 1023002), meta=np.ndarray>
    xice                    (time, nCells) float32 168MB dask.array<chunksize=(1, 1023002), meta=np.ndarray>
    sh2o                    (time, nCells, nSoilLevels) float32 671MB dask.array<chunksize=(1, 3226388, 1), meta=np.ndarray>
    smois                   (time, nCells, nSoilLevels) float32 671MB dask.array<chunksize=(1, 3226388, 1), meta=np.ndarray>
    tslb                    (time, nCells, nSoilLevels) float32 671MB dask.array<chunksize=(1, 3226388, 1), meta=np.ndarray>
    xtime                   (time, StrLen) |S1 64B dask.array<chunksize=(1, 64), meta=np.ndarray>
Attributes: (12/114)
    model_name:                        mpas
    core_name:                         atmosphere
    source:                            MPAS
    Conventions:                       MPAS
    git_version:                       v6.1-dirty
    on_a_sphere:                       YES
    ...                                ...
    config_oml_relaxation_time:        864000.0
    config_sounding_interval:          none
    file_id:                           hawfo76vlx
    NCO:                               netCDF Operators version 4.9.5 (Homepa...
    history:                           Wed Mar 24 19:15:02 2021: ncks -A -v x...
    history_of_appended_files:         Wed Mar 24 19:15:02 2021: Appended fil...

In [46]:
ds_mpas = xr.open_dataset(fil, engine='netcdf4', mask_and_scale=True, chunks={'Time': 'auto'})


In [48]:
if len(ds_mpas['Time']) > 1:
    print("Multiple times detected... will probably break preprocessor")

if isinstance(meshfil, xr.Dataset):
    ds_static = meshfil
elif isinstance(meshfil, Path):
    ds_static = xr.open_dataset(meshfil)
else:
    raise ValueError("meshfil needs to be a dataset or a path")

# Use a fixed reference date for all files
ref_date = '2000-01-01 00:00:00'  # Or any other suitable fixed date

# Clean and convert xtime strings
time_str = ds_mpas.xtime.astype(str).values.astype('U').ravel()
    
print(f"{time_str = }")

# Remove extra whitespace and handle empty strings
time_str = [x.strip() for x in time_str]
time_str = [x.replace("_", " ") for x in time_str]
    
if isinstance(time_str, np.ndarray) or isinstance(time_str, list):
    time_str = "".join(time_str)

print(f"{time_str = }")



time_str = array(['2', '0', '1', '6', '-', '0', '8', '-', '0', '1', '_', '0', '0',
       ':', '0', '0', ':', '0', '0', ' ', ' ', ' ', ' ', ' ', ' ', ' ',
       ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ',
       ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ',
       ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
      dtype='<U1')
time_str = '2016-08-01 00:00:00'
